# Lab Supply Calculator

Welcome! Use this tool to determine the quantity of supplies you will need for a given sample size, compare that to the current inventory list, generate a report of supply needs and reorder status, and then update the inventory list with the supply needs. This tool requires access to the inventory list (a shared Google sheet), a lab protocol (saved in GitHub and formatted in yaml), and a user-defined sample size (number of samples and plates).
<br>
<br>
Please make sure you have access to the Inventory Google sheet prior to starting. The inventory list is located at the following link: https://docs.google.com/spreadsheets/d/1I1q2VBpyYtZItV5uP4WE2zGC5sm3qWFAK2D8yQtntFg/edit?usp=sharing.
<br>
<br>
The protocols are currently being stored in the following repository: https://github.com/aono87/inventory_test/tree/main/test_protocol_check/protocols
<br>
<br>
If you are unable to access the sheet, please contact Vicki Pease (vicki.pease@noaa.gov).

If you have any problem running the program, please contact Aubrie Onoufriou (aubrie.onoufriou@noaa.gov)
<br>
<br>

# Instructions
1. Press "Run all" to get started.

    Note: It may take several seconds to install and update the required packages.

    Note: You will get a pop up to authenticate yourself the first time this is run.

2. Enter your name (first initial and last name, e.g., AOnoufriou).

3. Enter the number of samples and number of plates.

4. Choose the protocol from the dropdown menu.

5. Choose the associated project from the drop down menu (this corresponds to active projects on the SWFSC MMTD Genetics GitHub Repo)

6. Click "Generate Report".

7. When you are happy with the report, you can save the report to your computer as a CSV by clicking "Download CSV".

8. Press "Update Inventory Sheet" to add the required amounts, protocol and user information to the Inventory Google sheet document.

9. If there is insufficient stock available for any item, please copy the information provided and email Vicki for re-ordering.

**Note about Google credentials:**
This notebook is run through the Google environment and therefore will use your same NOAA Google credentials. As long as you have permission to access the "Inventory" Google sheet, this program will run.

**Troubleshooting:**
You can clear all output and start clean by going to "Edit", then "Clear all outputs".
Sometimes the program gets clunky when you update the report several times. This can cause two CSV reports to download. If this starts to happen, clear everything (include the packages that have been installed) by going to "Runtime", then  "Disconnect and delete runtime". You can then try again by pressing  "Run all".

In [ ]:
#@title Install packages and imports
##Install packages and imports
# Installing packages
import subprocess
import sys
import re

def install_package(package):
    # Use --upgrade to ensure the latest version
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package])

try:
    import yaml
except ImportError:
    install_package("pyyaml")
import yaml

try:
    import gspread
    # FIX: Add explicit upgrade for gspread
    install_package("gspread") # This now includes --upgrade
except ImportError:
    install_package("gspread")

# Ensure these are installed for Google Colab authentication with gspread
try:
    import google.auth
except ImportError:
    install_package("google-auth")
try:
    import google.auth.transport.requests
except ImportError:
    install_package("google-auth-httplib2")

# Install the pytz library for timezone handling
try:
    import pytz
except ImportError:
    install_package("pytz")
import pytz


# Import for Google Colab authentication
try:
    from google.colab import auth
except ImportError:
    print("Warning: google.colab.auth not found. Running outside Colab might require manual authentication setup for gspread.")


# Import all required libraries
import requests
import csv
import pandas as pd
import os
import math # Added for ceiling function
from io import StringIO
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
from datetime import datetime # Import the datetime object


print("✅ All packages installed and imported successfully!")


# --- Google Sheet Authentication Sanity Check ---
global gc # Make gc global so it can be used in load_inventory
try:
    print("\n🔑 Authenticating Google Colab for Sheets access (this may open a pop-up)...")
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets authentication successful!")
except Exception as e:
    print(f"❌ Initial Google Sheets authentication failed: {e}")
    print("Please ensure you complete the authentication pop-up and have proper sheet permissions.")
    gc = None # Set gc to None if authentication fails
# ----------------------------------------------------------

# Define GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
PROTOCOL_DIR = "test_protocol_check/protocols"
GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1I1q2VBpyYtZItV5uP4WE2zGC5sm3qWFAK2D8yQtntFg/edit?usp=sharing"

print("✅ Configuration set")

#Helper Functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path):
    url = github_raw_url(path)
    try:
        r = requests.get(url)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {path}: {e}")
        raise

def get_protocol_list(user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    try:
        r = requests.get(url)
        r.raise_for_status()
        files = r.json()
        # Filter for files ending with .yaml
        protocol_files = [f['path'] for f in files if f['name'].endswith('.yaml') and f['type'] == 'file']
        return protocol_files
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching protocol list: {e}")
        return []


def get_project_list(user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    try:
        r = requests.get(url)
        r.raise_for_status() # Raise an exception for bad status codes (like 404 Not Found)
        files = r.json()
        # Assuming project names are the names of the directories within the 'active' folder
        # Filter for directories
        project_names = [f['name'] for f in files if f['type'] == 'dir']
        if not project_names:
             # If no directories found, check if the 'active' folder itself exists and is accessible
             # This would require another API call or checking the error status specifically
             # For now, rely on the r.raise_for_status() above to indicate if the path is bad.
             # If the path is good but no directories, the list will be empty.
             print(f"⚠️ Warning: Found folder '{path}' but no project sub-folders (directories) within it.")
        return project_names
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching project list: {e}")
        # More specific error messages based on status code could be added here if needed
        if r.status_code == 404:
             print(f"❌ Could not find folder '{path}' in the repository root.")
        return [] # Return empty list on error


def load_inventory():
    if gc is None:
        print("❌ Google Sheets client not initialized. Cannot load inventory.")
        return pd.DataFrame() # Return empty DataFrame on failure
    try:
        spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
        worksheet = spreadsheet.get_worksheet(0)
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)

        # Clean up column names to prevent errors
        df.columns = df.columns.str.strip()

        # Ensure numeric columns are treated as numbers, handling empty strings
        # Include the new 'Stock Unit Quantity-Live' column
        numeric_cols = ['Amount per stock unit (In protocol unit)', 'Stock Unit Quantity', 'Stock Unit Quantity-Live']
        for col in numeric_cols:
            # Check if the column exists before trying to convert
            if col in df.columns:
                 df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            else:
                 print(f"⚠️ Warning: Column '{col}' not found in the inventory sheet.")
                 # Add the column with default 0 if it's missing to prevent errors later
                 df[col] = 0


        print("✅ Inventory loaded successfully from Google Sheet into DataFrame.")
        return df
    except Exception as e:
        print(f"❌ Error loading inventory from Google Sheet: {e}")
        return pd.DataFrame()

def load_protocol(text):
    try:
        protocol_data = yaml.safe_load(text)
        return protocol_data
    except Exception as e:
        print(f"❌ Error loading protocol: {e}")
        return {}

def calculate_needs(protocol_data, sample_count, plate_count):
    """Calculates total needs and returns a DataFrame, preserving duplicate items."""
    needs_list = []

    # Handle per-sample supplies
    # Assume supplies_per_sample is a list of dictionaries if duplicates are intended
    sample_supplies = protocol_data.get('supplies_per_sample')
    if isinstance(sample_supplies, list):
        for item_details in sample_supplies:
            # Ensure item_details is a dictionary and has 'item' and 'quantity' keys
            if isinstance(item_details, dict) and 'item' in item_details and 'quantity' in item_details:
                item = item_details['item']
                qty = item_details['quantity']
                unit = item_details.get('unit', 'N/A')
                needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/sample",
                    'Total needed (protocol unit)': qty * sample_count,
                    'protocol_unit': unit
                })
            elif isinstance(item_details, dict) and len(item_details) == 1:
                 # Handle case where format is just item_name: quantity/unit
                 item, details = list(item_details.items())[0]
                 if isinstance(details, dict):
                     qty = details.get('quantity', details)
                     unit = details.get('unit', 'N/A')
                 else:
                     qty = details
                     unit = 'N/A'
                 needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/sample",
                    'Total needed (protocol unit)': qty * sample_count,
                    'protocol_unit': unit
                 })

    elif isinstance(sample_supplies, dict): # Handle existing dictionary format
        for item, details in sample_supplies.items():
            qty = details.get('quantity', details) if isinstance(details, dict) else details
            unit = details.get('unit', 'N/A') if isinstance(details, dict) else 'N/A'
            # Append details for each item from the protocol
            needs_list.append({
                'Item': item,
                'Required per unit': f"{qty} {unit}/sample",
                'Total needed (protocol unit)': qty * sample_count,
                'protocol_unit': unit
            })


    # Handle per-plate supplies
    # Assume supplies_per_plate is a list of dictionaries if duplicates are intended
    plate_supplies = protocol_data.get('supplies_per_plate')
    if isinstance(plate_supplies, list):
        for item_details in plate_supplies:
             # Ensure item_details is a dictionary and has 'item' and 'quantity' keys
            if isinstance(item_details, dict) and 'item' in item_details and 'quantity' in item_details:
                item = item_details['item']
                qty = item_details['quantity']
                unit = item_details.get('unit', 'N/A')
                needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/plate",
                    'Total needed (protocol unit)': qty * plate_count,
                    'protocol_unit': unit
                })
            elif isinstance(item_details, dict) and len(item_details) == 1:
                 # Handle case where format is just item_name: quantity/unit
                 item, details = list(item_details.items())[0]
                 if isinstance(details, dict):
                     qty = details.get('quantity', details)
                     unit = details.get('unit', 'N/A')
                 else:
                     qty = details
                     unit = 'N/A'
                 needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/plate",
                    'Total needed (protocol unit)': qty * plate_count,
                    'protocol_unit': unit
                 })

    elif isinstance(plate_supplies, dict): # Handle existing dictionary format
        for item, details in plate_supplies.items():
            qty = details.get('quantity', details) if isinstance(details, dict) else details
            unit = details.get('unit', 'N/A') if isinstance(details, dict) else 'N/A'
            # Append details for each item from the protocol
            needs_list.append({
                'Item': item,
                'Required per unit': f"{qty} {unit}/plate",
                'Total needed (protocol unit)': qty * plate_count,
                'protocol_unit': unit
            })


    if not needs_list:
        print("❌ Protocol missing 'supplies_per_sample' and/or 'supplies_per_plate' section or they are empty/malformed.")
        return pd.DataFrame() # Return empty DataFrame if no needs are calculated

    # Return DataFrame from the list of needs, which inherently handles duplicates with different rows
    return pd.DataFrame(needs_list)


# Removed the debug print for Helper functions defined

In [ ]:
# @title Generate Supply Report & Update Sheet
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import os
import csv
import gspread
from datetime import datetime
import pytz
import math
import numpy as np

# This is the "initialize-once" pattern.
# The code inside this 'if' block will only run ONCE per session,
# preventing event handlers from stacking up on re-runs.
if 'calculator_app' not in globals():
    print("🚀 Initializing Lab Supply Calculator for the first time...")
    print("=" * 50)

    # --- Step 1: Create all widgets just once ---
    name_input = widgets.Text(value='AOnoufriou', placeholder='e.g., AOnoufriou', description='Your Name:', style={'description_width': 'initial'})
    samples_input = widgets.IntText(value=96, description='Number of Samples:', style={'description_width': 'initial'})
    plates_input = widgets.IntText(value=1, description='Number of Plates:', style={'description_width': 'initial'})
    protocol_dropdown = widgets.Dropdown(description='Protocol:', layout=widgets.Layout(width='95%'), style={'description_width': 'initial'})

    # Create the Project dropdown widget
    project_dropdown = widgets.Dropdown(description='Project:', layout=widgets.Layout(width='95%'), style={'description_width': 'initial'})


    generate_button = widgets.Button(description='Generate Report', button_style='primary', tooltip='Click to generate the supply report', layout=widgets.Layout(width='200px', margin='0 10px 0 0'))
    update_sheet_button = widgets.Button(description='Update Inventory Sheet', button_style='warning', tooltip='Writes the hold information to new columns in the Google Sheet', layout=widgets.Layout(width='200px', margin='0 10px 0 0', display='none'))
    download_button = widgets.Button(description='Download CSV', button_style='success', tooltip='Click to download the report as a CSV file', layout=widgets.Layout(width='200px', display='none'))
    output_area = widgets.Output()

    # This dictionary will live inside the app structure
    report_data_for_csv = {}

    # --- Step 2: Define all handler functions just once ---
    def generate_report(b):
        """Calculates and displays the supply report using a merged DataFrame."""
        with output_area:
            output_area.clear_output(wait=True)
            update_sheet_button.layout.display = 'none'
            download_button.layout.display = 'none'
            generate_button.disabled = True
            generate_button.description = "Generating..."
            try:
                print("🔄 Generating report...")
                # 1. Load data into DataFrames
                inventory_df = load_inventory()

                if inventory_df.empty:
                    raise ValueError("Inventory data could not be loaded. Please check logs for a Google Sheet error.")

                # The protocol_dropdown value is now just the name, need to reconstruct the full path
                selected_protocol_name = protocol_dropdown.value
                full_protocol_path = f"{PROTOCOL_DIR}/{selected_protocol_name}.yaml"


                protocol_text = fetch_file(full_protocol_path)
                protocol = load_protocol(protocol_text)
                needs_df = calculate_needs(protocol, samples_input.value, plates_input.value)

                if needs_df.empty:
                     print("❌ Protocol data could not be loaded or protocol is empty.")
                     return # Exit function if needs_df is empty

                print(f"✅ Calculated needs for {len(needs_df)} items")

                # 2. Merge the two DataFrames
                merged_df = pd.merge(needs_df, inventory_df, on='Item', how='left')

                # 3. Perform calculations on the merged DataFrame
                merged_df['Amount per stock unit (In protocol unit)'] = pd.to_numeric(merged_df['Amount per stock unit (In protocol unit)'], errors='coerce').fillna(0)
                merged_df['Stock Unit Quantity-Live'] = pd.to_numeric(merged_df['Stock Unit Quantity-Live'], errors='coerce').fillna(0)

                merged_df['Total needed (stock unit)'] = np.where(
                    merged_df['Amount per stock unit (In protocol unit)'] > 0,
                    merged_df['Total needed (protocol unit)'] / merged_df['Amount per stock unit (In protocol unit)'],
                    np.nan
                )

                merged_df['Sufficient Stock'] = np.where(
                    merged_df['Stock Unit Quantity-Live'] >= merged_df['Total needed (stock unit)'], 'YES', 'NO'
                )
                merged_df['Sufficient Stock'] = merged_df['Sufficient Stock'].fillna('MISSING/N/A')

                # 4. Format columns for display
                merged_df['Total needed (protocol unit)'] = merged_df.apply(
                    lambda row: f"{round(row['Total needed (protocol unit)'], 3)} {row.get('protocol_unit', 'Unit')}" if pd.notna(row['Total needed (protocol unit)']) else "N/A", axis=1
                )
                merged_df['Total needed (stock unit)'] = merged_df.apply(
                    lambda row: f"{round(row['Total needed (stock unit)'], 3)} {row.get('Stock unit', 'Unit')}(s)" if pd.notna(row['Total needed (stock unit)']) else "N/A", axis=1
                )
                merged_df['Stock on hand'] = merged_df.apply(
                    lambda row: f"{round(row['Stock Unit Quantity-Live'], 3)} {row.get('Stock unit', 'Unit')}(s)" if pd.notna(row['Stock Unit Quantity-Live']) else "N/A", axis=1
                )

                merged_df.rename(columns={'Expiration Date': 'Expiration date'}, inplace=True)

                # 5. Select and order columns for the final report
                report_columns = [
                    "Item", "Required per unit", "Total needed (protocol unit)",
                    "Total needed (stock unit)", "Stock on hand",
                    "Sufficient Stock", "Expiration date"
                ]
                final_report_df = merged_df[report_columns].fillna('N/A')

                # Store original data for download/update functions
                report_data_for_csv['df'] = final_report_df
                report_data_for_csv['user_name'] = name_input.value
                report_data_for_csv['protocol_name'] = selected_protocol_name
                report_data_for_csv['sample_size'] = samples_input.value
                report_data_for_csv['plate_count'] = plates_input.value
                report_data_for_csv['project_name'] = project_dropdown.value

                print(f"\n📋 Supply Report for {samples_input.value} samples and {plates_input.value} plates by {name_input.value}")
                print(f"Protocol: {report_data_for_csv['protocol_name']}")
                print(f"Project: {report_data_for_csv['project_name']}")
                print("=" * 80)

                # 6. Create Insufficient Stock Summary
                insufficient_df = final_report_df[final_report_df['Sufficient Stock'] == 'NO']
                summary_widgets = []
                if not insufficient_df.empty:
                    summary_header = widgets.HTML("<b><font color='red'>⚠️ Insufficient Stock Alert:</font></b>")
                    summary_widgets.append(summary_header)
                    for _, row in insufficient_df.iterrows():
                        item_name = row['Item']
                        needed = row['Total needed (stock unit)']
                        on_hand = row['Stock on hand']
                        summary_line = widgets.HTML(f"&nbsp;&nbsp;&nbsp;• <b>{item_name}</b>: Needed {needed}, On Hand {on_hand}")
                        summary_widgets.append(summary_line)

                    # Add the ordering instruction line
                    ordering_instruction = widgets.HTML("<i style='font-size:14px; padding-top: 8px; display: block;'>Please copy and paste this information in an email to Vicki (vicki.pease@noaa.gov) for ordering.</i>")
                    summary_widgets.append(ordering_instruction)

                summary_vbox = widgets.VBox(summary_widgets)

                # 7. Create the interactive report with checkboxes
                select_all_cb = widgets.Checkbox(value=True, description="Select/Deselect All")
                individual_checkboxes = []

                col_layouts = [
                    widgets.Layout(width='50px'),
                    widgets.Layout(width='250px', margin='0 0 0 10px'), # Added margin to create space
                    widgets.Layout(width='150px'),
                    widgets.Layout(width='150px'),
                    widgets.Layout(width='150px'),
                    widgets.Layout(width='150px'),
                    widgets.Layout(width='120px'),
                    widgets.Layout(width='120px')
                ]

                header_labels = [widgets.HTML(value="<b>Update?</b>")] + [widgets.HTML(value=f"<b>{col}</b>") for col in final_report_df.columns]
                header_box = widgets.HBox(header_labels)

                for i, label in enumerate(header_box.children):
                    label.layout = col_layouts[i]

                report_rows = [header_box]

                for index, row in final_report_df.iterrows():
                    cb = widgets.Checkbox(value=True, indent=False)
                    individual_checkboxes.append(cb)
                    row_labels = [widgets.Label(value=str(value)) for value in row]
                    row_box = widgets.HBox([cb] + row_labels)
                    for i, item in enumerate(row_box.children):
                        item.layout = col_layouts[i]
                    report_rows.append(row_box)

                def on_select_all_change(change):
                    for cb in individual_checkboxes:
                        cb.value = change['new']
                select_all_cb.observe(on_select_all_change, names='value')

                report_data_for_csv['checkboxes'] = individual_checkboxes
                report_vbox = widgets.VBox(report_rows)

                # Display the summary, controls, and report
                display(widgets.VBox([summary_vbox, select_all_cb, report_vbox]))

                update_sheet_button.disabled = False
                update_sheet_button.description = 'Update Inventory Sheet'
                update_sheet_button.layout.display = ''
                download_button.disabled = False
                download_button.description = 'Download CSV'
                download_button.layout.display = ''
            except Exception as e:
                print(f"❌ Error during report generation: {e}")
            finally:
                generate_button.disabled = False
                generate_button.description = "Generate Report"

    def update_inventory_sheet(b):
        """Writes the hold information to the Google Sheet with correct timezone."""
        if b.disabled: return
        b.disabled = True
        with output_area:
            print("\n🔄 Updating Google Sheet with hold information...")
            b.description = "Updating..."
            try:
                report_df = report_data_for_csv.get('df')
                checkboxes = report_data_for_csv.get('checkboxes', [])
                user_name = report_data_for_csv.get('user_name', 'N/A')
                protocol_name = report_data_for_csv.get('protocol_name', 'N/A')
                project_name = report_data_for_csv.get('project_name', 'N/A')

                spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
                worksheet = spreadsheet.get_worksheet(0)

                headers = worksheet.row_values(1)
                try:
                    next_col_index = headers.index('')
                    next_col = next_col_index + 1
                    print(f"✅ Found first empty column at index {next_col_index}.")
                except ValueError:
                    next_col = len(headers) + 1
                    print(f"✅ No empty columns found in header row. Appending to the end (Column {gspread.utils.rowcol_to_a1(1, next_col)}).")

                pacific_tz = pytz.timezone('America/Los_Angeles')
                now_pacific = datetime.now(pacific_tz)
                date_str = now_pacific.strftime('%Y-%m-%d %I:%M %p %Z')

                new_header = f"{user_name} - {project_name} - {protocol_name} - {date_str}"
                header_update_range = f'{gspread.utils.rowcol_to_a1(1, next_col)}'
                worksheet.update(range_name=header_update_range, values=[[new_header]])
                print(f"✅ Added new header in column {gspread.utils.rowcol_to_a1(1, next_col)}.")

                inventory_data = worksheet.get_all_records()
                item_to_row_map = {str(record['Item']).strip(): i + 2 for i, record in enumerate(inventory_data)}

                updates = []
                items_to_update_count = 0
                for idx, row in report_df.iterrows():
                    # Only proceed if the checkbox for this row is ticked
                    if checkboxes and idx < len(checkboxes) and checkboxes[idx].value:
                        items_to_update_count += 1
                        item_name = str(row['Item']).strip()
                        needed_str = str(row['Total needed (stock unit)'])

                        value_to_write = 'N/A'
                        if pd.notna(needed_str) and needed_str != 'N/A':
                            numeric_part_str = needed_str.split(' ')[0]
                            try:
                                value_to_write = float(numeric_part_str)
                            except (ValueError, TypeError):
                                value_to_write = numeric_part_str

                        if item_name in item_to_row_map:
                            row_idx = item_to_row_map[item_name]
                            updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, next_col)}', 'values': [[value_to_write]]})
                        else:
                             print(f"🤷 No matching item '{item_name}' found in the inventory sheet to update.")

                if updates:
                    worksheet.batch_update(updates, value_input_option='USER_ENTERED')
                    print(f"✅ Successfully updated hold information for {len(updates)} items in column {gspread.utils.rowcol_to_a1(1, next_col)}.")
                    b.description = 'Updated ✓'
                elif items_to_update_count == 0:
                    print("🤷 No items were selected for update.")
                    b.description = 'Nothing to Update'
                else:
                    print("🤷 No selected items matched the inventory sheet. Nothing to update.")
                    b.description = 'Nothing to Update'
            except Exception as e:
                print(f"❌ Sheet update failed: {e}")
                b.disabled = False
                b.description = 'Update Failed'


    def download_csv(b):
        """Creates and downloads a filtered report based on ticked checkboxes."""
        if b.disabled: return
        b.disabled = True
        with output_area:
            print("\n💾 Preparing CSV for download...")
            b.description = "Downloading..."
            try:
                # Retrieve the full dataframe AND the checkboxes
                df = report_data_for_csv.get('df')
                checkboxes = report_data_for_csv.get('checkboxes', [])

                # Create a boolean list (mask) from the checkbox values and filter the DataFrame
                if checkboxes and len(checkboxes) == len(df):
                    ticked_mask = [cb.value for cb in checkboxes]
                    filtered_df = df[ticked_mask].copy()
                else:
                    # Fallback to using the full dataframe if something is wrong
                    filtered_df = df.copy()

                user_name = report_data_for_csv.get('user_name', 'N/A')
                protocol_name = report_data_for_csv.get('protocol_name', 'N/A')
                sample_size = report_data_for_csv.get('sample_size', 'N/A')
                plate_count = report_data_for_csv.get('plate_count', 'N/A')
                project_name = report_data_for_csv.get('project_name', 'N/A')

                protocol_base_name = os.path.splitext(protocol_name)[0].replace(' ', '_')

                pacific_tz = pytz.timezone('America/Los_Angeles')
                now_pacific = datetime.now(pacific_tz)
                date_str = now_pacific.strftime('%Y-%m-%d')
                time_str = now_pacific.strftime('%Y-%m-%d %I:%M:%S %p %Z')

                filename = f"Report_{protocol_base_name}_{sample_size}-samples_{plate_count}-plates_{user_name}_{date_str}.csv"

                metadata_rows = {
                    'Item': [
                        f"User: {user_name}", f"Protocol: {protocol_name}",
                        f"Sample Size: {sample_size}", f"Plate Count: {plate_count}",
                        f"Project: {project_name}", f"Generated: {time_str}"
                    ]
                }

                # Use the filtered_df for column info if it's not empty, otherwise use original df
                cols_df = filtered_df if not filtered_df.empty else df
                for col in cols_df.columns:
                    if col != 'Item':
                        metadata_rows[col] = [''] * len(metadata_rows['Item'])

                metadata = pd.DataFrame(metadata_rows)
                # Use the filtered_df for the final output
                final_output = pd.concat([metadata, filtered_df], ignore_index=True)
                final_output.to_csv(filename, index=False)

                print(f"📁 Starting download for: {filename}")
                files.download(filename)
                print("🎉 Download complete!")
                b.description = 'Downloaded ✓'
            except Exception as e:
                print(f"❌ Download failed: {e}")
                b.disabled = False
                b.description = "Download Failed"

    # --- Step 3: Connect handlers and populate dropdown just once ---
    generate_button.on_click(generate_report)
    update_sheet_button.on_click(update_inventory_sheet)
    download_button.on_click(download_csv)

    with output_area:
        try:
            print("🔍 Fetching available protocols from GitHub...")
            protocols = get_protocol_list(GITHUB_USER, REPO_NAME, PROTOCOL_DIR, BRANCH)
            if not protocols: raise ValueError("No protocols found on GitHub.")
            protocol_names = [os.path.splitext(os.path.basename(p))[0] for p in protocols]
            protocol_dropdown.options = protocol_names
            print(f"✅ Found {len(protocol_names)} protocols.")
        except Exception as e:
            print(f"❌ {e} Using a default.")
            protocol_dropdown.options = ["default_protocol"]

        try:
            print("🔍 Fetching available projects from GitHub...")
            project_user = "aono87"
            project_repo = "inventory_test"
            project_path = "test_protocol_check/projects/active"
            projects = get_project_list(project_user, project_repo, project_path, BRANCH)
            if not projects: raise ValueError("No projects found on GitHub.")
            project_dropdown.options = ["No Assigned Project"] + projects
            print(f"✅ Found {len(projects)} projects and added 'No Assigned Project' option.")
        except Exception as e:
            print(f"❌ {e} Using a default.")
            project_dropdown.options = ["No Assigned Project", "Default Project"]


    # --- Step 4: Assemble the UI and store it in a global variable ---
    buttons_box = widgets.HBox([generate_button, update_sheet_button, download_button])
    globals()['calculator_app'] = widgets.VBox([
        widgets.HTML("<b>Please enter your details below and click 'Generate Report'.</b>"),
        name_input,
        samples_input,
        plates_input,
        protocol_dropdown,
        project_dropdown,
        buttons_box,
        widgets.HTML("<hr>"),
        output_area
    ])

# --- Display the App ---
display(globals()['calculator_app'])